In [1]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-jizgmnwi
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-jizgmnwi
  Resolved https://github.com/unslothai/unsloth.git to commit b4c48d9c5e78203909495bf9beaa29a5c9aeaeeb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.5-py3-none-any.whl size=176840 sha256=b751484c1916c6b774d2fae5c6569700ea39a7d3b5dd58338dce9aa068022d85
  Stored in directory: /tmp/pip-ephem-wheel-cache-03d02dij/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.1.5
    Uninstalling unsloth-2025.1.5:
      Successfully uninstalled unsloth-2025.1.5


In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from datasets import load_dataset
dataset = load_dataset("code_search_net", split = "train")


In [5]:
dataset_example = dataset[0]

In [6]:
dataset_example

{'repository_name': 'adamrehn/ue4cli',
 'func_path_in_repository': 'ue4cli/Utility.py',
 'func_name': 'Utility.findArgs',
 'whole_func_string': 'def findArgs(args, prefixes):\n\t\t"""\n\t\tExtracts the list of arguments that start with any of the specified prefix values\n\t\t"""\n\t\treturn list([\n\t\t\targ for arg in args\n\t\t\tif len([p for p in prefixes if arg.lower().startswith(p.lower())]) > 0\n\t\t])',
 'language': 'python',
 'func_code_string': 'def findArgs(args, prefixes):\n\t\t"""\n\t\tExtracts the list of arguments that start with any of the specified prefix values\n\t\t"""\n\t\treturn list([\n\t\t\targ for arg in args\n\t\t\tif len([p for p in prefixes if arg.lower().startswith(p.lower())]) > 0\n\t\t])',
 'func_code_tokens': ['def',
  'findArgs',
  '(',
  'args',
  ',',
  'prefixes',
  ')',
  ':',
  'return',
  'list',
  '(',
  '[',
  'arg',
  'for',
  'arg',
  'in',
  'args',
  'if',
  'len',
  '(',
  '[',
  'p',
  'for',
  'p',
  'in',
  'prefixes',
  'if',
  'arg',
  '

In [7]:
eval_dataset = load_dataset("code_search_net", split = "validation")

In [10]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
"Generate a {} function's code based on its documentation"

### Input:
Code Prompt: {}

### Response:
Code: {}

"""

EOS_TOKEN = tokenizer.eos_token

def formating_propmpt_func(examples):
  languages = examples["language"]
  docs = examples["func_documentation_string"]
  codes = examples["func_code_string"]
  texts = []
  for language,doc,code in zip(languages,docs,codes):
    text = alpaca_prompt.format(language,doc,code) + EOS_TOKEN
    texts.append(text)

  return {"text":texts,}

dataset = dataset.map(formating_propmpt_func,batched = True,)
eval_dataset = eval_dataset.map(formating_propmpt_func,batched = True,)

Map:   0%|          | 0/89154 [00:00<?, ? examples/s]

In [17]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

save_path = "outputs/trained_model_llama3"
trainer.train()
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,880,853 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.169200
2,0.174800
3,0.128600
4,0.187300
5,0.179900
6,0.351200
7,0.228000
8,0.143000
9,0.178100
10,0.178500


Model and tokenizer saved to outputs/trained_model_llama3


In [25]:
dataset[651]["func_documentation_string"]

'Fetch the status of submitted athena query. Returns None or one of valid query states.\n\n        :param query_execution_id: Id of submitted athena query\n        :type query_execution_id: str\n        :return: str'

In [26]:
model = FastLanguageModel.for_inference(model)
def generate_code(language,code_prompt):

    device = model.device
    formatted_prompt = f"Instruction: Generate a {language} function's code based on its documentation.\nCode Prompt: {code_prompt}\nCode:"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_length=1000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test Code generation
dataset = load_dataset("code_search_net", split = "test")

dataset_example= dataset[651]
context_example = dataset_example["func_documentation_string"]
language_example = dataset_example["language"]

print("Generated Code:", generate_code(language_example,context_example))

Generated Code: Instruction: Generate a python function's code based on its documentation.
Code Prompt: Fetch the status of submitted athena query. Returns None or one of valid query states.

        :param query_execution_id: Id of submitted athena query
        :type query_execution_id: str
        :return: str
Code: def _get_athena_query_status(self, query_execution_id):
        """
        Fetch the status of submitted athena query. Returns None or one of valid query states.

        :param query_execution_id: Id of submitted athena query
        :type query_execution_id: str
        :return: str
        """
        query = self._client.get_query_execution(
            QueryExecutionId=query_execution_id
        )
        return query.get("QueryExecution", {}).get("Status", {}).get("State")


